In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import imageio

from scipy.optimize import basinhopping

from pymedphys.film import calc_calibration_points, load_cal_scans, create_dose_function, calc_net_od

In [ ]:
data_dir = Path('../../packages/pymedphys_analysis/tests/film/data/spine_case/DatasetA')

In [ ]:
prescans_dir = data_dir.joinpath('prescans/calibration')
postscans_dir = data_dir.joinpath('postscans/calibration')

In [ ]:
treatment_prescan = data_dir.joinpath('prescans/treatment.tif')
treatment_postscan = data_dir.joinpath('postscans/treatment.tif')

In [ ]:
prescans = load_cal_scans(prescans_dir)
postscans = load_cal_scans(postscans_dir)

In [ ]:
alignments = {650.0: (-0.2603466805472101, -0.05536224482995437, -0.07369560088369552),
 200.0: (0.01989287473874566, 0.05653957204379942, 0.01958896784090862),
 1250.0: (-0.030984185305909597, -0.013095484617055644, -0.7892509138699312),
 1000.0: (0.014882233322958316, 0.047914479072576996, -0.4978947226999769),
 400.0: (-0.10014108696626364, -0.0505205399216578, 0.35312224601205056),
 800.0: (-0.36114295467922, -0.24265504603088778, -3.0572273897726703),
 0.0: (0.059460648677009, 0.011342191873970335, 0.5105895559969915),
 550.0: (0.12412455554436072, -0.06665129104699223, -1.804139263417845),
 300.0: (-0.011790177683326901, -0.18416361576439752, -2.727632477227316),
 1400.0: (-0.133020484721831, 0.21720646311583633, -1.774500274342223),
 700.0: (0.012808244265859131, -0.09308716649676804, -0.3799893464300393),
 620.0: (-0.23915761204525535, 0.06496677146286638, -0.1765544707738022),
 500.0: (-0.2019659941820111, -0.2185103120976351, 1.3280398730215275),
 900.0: (0.07055237770009783, 0.5036890077460608, 0.5275840400344339),
 600.0: (0.08945839395271547, -0.01098711847707867, 0.8744853354710658),
 750.0: (-0.0005724882445936516, -0.24266847067130143, -1.3064618106060368),
 580.0: (0.08104532576324767, 0.1622561143167344, -0.7394495101711168),
 1100.0: (-0.07894465096909696, 0.12901876195049808, -0.46155730418785496)}

# alignments = None  # Uncomment this line to recalc the alignments (slow)

In [ ]:
points, alignments = calc_calibration_points(prescans, postscans, alignments=alignments)
# alignments

In [ ]:
dose_cal = []
net_od_cal = []

for key, item in points.items():
    net_od_cal.append(item)
    dose_cal.append(key)

In [ ]:
index = np.argsort(net_od_cal)
dose_cal = np.array(dose_cal)[index]
net_od_cal = np.array(net_od_cal)[index]

In [ ]:
dose_function = create_dose_function(net_od_cal, dose_cal)

In [ ]:
x = np.linspace(np.min(net_od_cal), np.max(net_od_cal))
y = dose_function(x)

plt.plot(net_od_cal, dose_cal, '.')
plt.plot(x, y)

In [ ]:
treatment_prescan = imageio.imread(treatment_prescan)
treatment_postscan = imageio.imread(treatment_postscan)

In [ ]:
treatment_alignment = None

In [ ]:
treatment_net_od, treatment_alignment = calc_net_od(
    treatment_prescan, treatment_postscan, alignment=treatment_alignment)
treatment_alignment

In [ ]:
treatment_dose = dose_function(treatment_net_od)